In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small") 

In [19]:
text = "임베딩 테스트를 하기 위한 샘플 문장입니다."

In [20]:
query_result = embeddings.embed_query(text)


In [22]:
query_result.__len__()

1536

In [23]:
doc_result = embeddings.embed_documents([text, text, text, text])


In [25]:
doc_result.__len__()

4

In [37]:
embeddings_1024 = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1024)
embeddings_original = OpenAIEmbeddings(model="text-embedding-3-small")

In [39]:
len_1024 = embeddings_1024.embed_documents([text])
len_original = embeddings_original.embed_documents([text])

In [44]:
print(len(len_1024[0]))
print(len(len_original[0]))


1024
1536


In [48]:
from sklearn.metrics.pairwise import cosine_similarity

In [49]:
sentence1 = "안녕하세요? 반갑습니다."
sentence2 = "안녕하세요? 반갑습니다!"
sentence3 = "안녕하세요? 만나서 반가워요."
sentence4 = "Hi, nice to meet you."
sentence5 = "I like to eat apples."

In [50]:
def similarity(a, b):
    return cosine_similarity([a], [b]) #

In [54]:
sentences = [sentence1, sentence2, sentence3, sentence4, sentence5] #

embedded_sentences = embeddings_1024.embed_documents(sentences) #
print("\n--- 유사도 계산 결과 ---")
for i, sentence_i in enumerate(embedded_sentences): #
    for j, other_sentence_j in enumerate(embedded_sentences): #
        if i < j: #
            print(f"[유사도 {similarity(sentence_i, other_sentence_j)}] {sentences[i]} <=====> {sentences[j]}") #


--- 유사도 계산 결과 ---
[유사도 [[0.96385287]]] 안녕하세요? 반갑습니다. <=====> 안녕하세요? 반갑습니다!
[유사도 [[0.84051826]]] 안녕하세요? 반갑습니다. <=====> 안녕하세요? 만나서 반가워요.
[유사도 [[0.50550912]]] 안녕하세요? 반갑습니다. <=====> Hi, nice to meet you.
[유사도 [[0.1379314]]] 안녕하세요? 반갑습니다. <=====> I like to eat apples.
[유사도 [[0.81426825]]] 안녕하세요? 반갑습니다! <=====> 안녕하세요? 만나서 반가워요.
[유사도 [[0.47912312]]] 안녕하세요? 반갑습니다! <=====> Hi, nice to meet you.
[유사도 [[0.13176721]]] 안녕하세요? 반갑습니다! <=====> I like to eat apples.
[유사도 [[0.5127969]]] 안녕하세요? 만나서 반가워요. <=====> Hi, nice to meet you.
[유사도 [[0.14082015]]] 안녕하세요? 만나서 반가워요. <=====> I like to eat apples.
[유사도 [[0.22497038]]] Hi, nice to meet you. <=====> I like to eat apples.


In [53]:
embedded_sentences.__len__()

5

In [55]:
from langchain.storage import LocalFileStore
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

In [56]:
embedding = OpenAIEmbeddings()

In [66]:
!pwd

/home/play/workspace


In [57]:
store = LocalFileStore("./cache/")

In [58]:
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embedding, # 기본 임베딩 모델 지정
    document_embedding_cache=store, # 로컬 저장소 지정
    namespace=embedding.model, # 임베딩 값 구분자 (생략 가능)
)

/home/play/miniconda3/envs/openai/lib/python3.12/site-packages/langchain/embeddings/cache.py:58: UserWarning: Using default key encoder: SHA‑1 is *not* collision‑resistant. While acceptable for most cache scenarios, a motivated attacker can craft two different payloads that map to the same cache key. If that risk matters in your environment, supply a stronger encoder (e.g. SHA‑256 or BLAKE2) via the `key_encoder` argument. If you change the key encoder, consider also creating a new cache, to avoid (the potential for) collisions with existing keys.
  _warn_about_sha1_encoder()


In [59]:
print(f"초기 캐시 키: {list(store.yield_keys())}")

초기 캐시 키: []


In [60]:
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = UnstructuredFileLoader("./data/ADsP - 정리.docx")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200) 


/tmp/ipykernel_193775/3071541522.py:4: LangChainDeprecationWarning: The class `UnstructuredFileLoader` was deprecated in LangChain 0.2.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-unstructured package and should be used instead. To use it run `pip install -U :class:`~langchain-unstructured` and import as `from :class:`~langchain_unstructured import UnstructuredLoader``.
  loader = UnstructuredFileLoader("./data/ADsP - 정리.docx")


In [ ]:
documents_spliter = text_splitter.split_documents(documents) #

In [63]:
%time db = FAISS.from_documents(documents_spliter, cached_embedder)

CPU times: user 83.4 ms, sys: 11.8 ms, total: 95.1 ms
Wall time: 2.02 s


In [64]:
%time db2 = FAISS.from_documents(documents_spliter, cached_embedder) #


CPU times: user 11 ms, sys: 5.55 ms, total: 16.5 ms
Wall time: 15.9 ms
